## 横軸 Force, 縦軸 Danger rate
Forceを変えながらDanger rateの変化のEmodのばらつきの影響をsobol指数で可視化する．

In [1]:
import sys
sys.path.append("/home/toroi/Document/fem_study/application")

import uncertainpy as un
import chaospy as cp                       # To create distributions
import numpy as np                         # For the time array
from scipy.integrate import odeint         # To integrate our equation

from myfem import LoadInput, \
    ComputeGlobalStiffness, \
    ApplyFixedDBCs, \
    SolveDisplacement, \
    PostProcessing

In [2]:
path_inputfile = "/home/toroi/Document/fem_study/HW01/hw01_input03.txt"

In [3]:
load_input = LoadInput(path_inputfile)
basename_without_ext = os.path.splitext(os.path.basename(path_inputfile))[0]
(material_info, nodal_info, elements_info, bc_info, ef_info, fd_info) =\
    load_input.load_txt()


In [4]:
cgs = ComputeGlobalStiffness(
    elements_info,
    nodal_info,
    material_info,
    ndim=2
)

K = cgs.run()
print('{:-^50}'.format(' End : Compute Global Stiffness '))
print("")

print('{:-^50}'.format(' Start : Apply Fixed DBCs '))
afdbc = ApplyFixedDBCs(bc_info, ef_info, fd_info)
mod_K = afdbc.modify_stiffness_matrix(K)
external_force_vector = afdbc.modify_external_force_vector(
    nodal_info.shape[0])

# print("modified K shape => ", mod_K.shape)
# print(mod_K)
# print("modified ef shape => ", external_force_vector.shape)
# print(external_force_vector)
print('{:-^50}'.format(' End : Apply Fixed DBCs '))
print("")

print('{:-^50}'.format(' Start : Solve Displacement '))
solve_dis_force = SolveDisplacement(mod_K, external_force_vector)
displacements = solve_dis_force.recover_global_displacements()
print('{:-^50}'.format(' End : Solve Displacement '))
print("")

print('{:-^50}'.format(' Start : Post Processing '))
pp = PostProcessing(
    elements_info,
    nodal_info,
    material_info,
    ndim=2
)

print('{:-^50}'.format(' Compute reaction forces '))
reaction_forces = pp.reaction_forces(K, displacements)

print('{:-^50}'.format(' Compute internal forces '))
inter_forces = pp.internal_forces(displacements)

print('{:-^50}'.format(' Create output file '))
pp.output_result("./results/" + basename_without_ext,
                 K,
                 displacements,
                 visualize=True)

print('{:-^50}'.format(' End : Post Processing '))


--------- End : Compute Global Stiffness ---------

------------ Start : Apply Fixed DBCs ------------
------------- End : Apply Fixed DBCs -------------

----------- Start : Solve Displacement -----------
------------ End : Solve Displacement ------------

------------ Start : Post Processing -------------
------------ Compute reaction forces -------------
------------ Compute internal forces -------------
--------------- Create output file ---------------
------------- End : Post Processing --------------


In [13]:
def uq_model_output_danger_rate(mat1_Emod, mat2_Emod):

    force_range = np.arange(-60, 60, 3)
    info = None

    cgs = ComputeGlobalStiffness(
        elements_info,
        nodal_info,
        material_info,
        ndim=2
    )

    # re-define Emod
    # print("Re-define Emod : ", Emod)
    material_info.loc[0, "Emod"] = mat1_Emod
    material_info.loc[1, "Emod"] = mat2_Emod

    danger_rates = []

    for force in force_range:
        # print("Force : ", force)
        ef_info.loc[0, "force"] = force

        K = cgs.run()

        afdbc = ApplyFixedDBCs(bc_info, ef_info, fd_info)
        mod_K = afdbc.modify_stiffness_matrix(K)
        external_force_vector = afdbc.modify_external_force_vector(
            nodal_info.shape[0])

        # print("modified K shape => ", mod_K.shape)
        # print(mod_K)
        # print("modified ef shape => ", external_force_vector.shape)
        # print(external_force_vector)
        solve_dis_force = SolveDisplacement(mod_K, external_force_vector)
        displacements = solve_dis_force.recover_global_displacements()
        pp = PostProcessing(
            elements_info,
            nodal_info,
            material_info,
            ndim=2
        )

        reaction_forces = pp.reaction_forces(K, displacements)

        inter_forces = pp.internal_forces(displacements)

        # modify 1d value for evaluation
        danger_rates.append(np.max(np.abs(inter_forces)))

    return force_range, danger_rates, info

In [19]:
Emod_dist1 = cp.Normal(700, 100)
Emod_dist2 = cp.Normal(20, 3)

param = {"mat1_Emod":Emod_dist1, "mat2_Emod":Emod_dist2}

model = un.Model(run=uq_model_output_danger_rate, labels=["Force", "Danger Rate"])

In [20]:
UQ = un.UncertaintyQuantification(model=model, parameters=param)

In [21]:
data = UQ.quantify(seed=10)

Re-define Emod :  Re-define Emod :  Re-define Emod : Re-define Emod : 353.9860247849597 412.745574558575
426.9795900078433
 Force : Force :  
 387.254425441425
-60Force : 
-60
Running model:   0%|          | 0/12 [00:00<?, ?it/s]

Force :  -57Force :  
-57
Force :  -57
Force :  -57
Force :  -54
Force :  -54
Force :  -54Force : 
 -54
Force :  -51
Force :  -51
Force :  -51Force : 
 -51
Force :  -48
Force :  -48
Force :  -48
Force :  -48
Force :  -45
Force :  -45
Force :  -45
Force :  -45
Force :  -42
Force :  -42
Force :  -42
Force :  -42
Force :  -39
Force :  -39
Force :  -39
Force :  -39
Force :  -36
Force :  -36
Force :  -36
Force :  -36
Force :  -33
Force :  -33
Force :  -33
Force :  -33
Force :  -30Force : 
 -30
Force :  -30
Force :  -30
Force :  -27
Force :  -27
Force :  -27
Force :  -27
Force :  -24
Force :  -24
Force :  -24
Force :  -24
Force :  -21
Force :  -21
Force :  Force : -21
 -21
Force :  -18
Force :  -18
Force :  -18
Force :  -18
Force :  -15
Force :  -15
Force :  -15
Fo

In [22]:
material_info

,mid,mtype,rho,Emod,A
0,1.0,1.0,1.0,700.0,10.0
1,2.0,1.0,1.0,20.0,10.0
